Glove tutorial  
py2.7  
linux
make is not supported under windows,one may use gensim-word2vector instead
PS:in the terminal of linux  '!' at the beginning should be omitted

# git glove

save code to document glove

In [1]:
!git clone https://github.com/stanfordnlp/GloVe glove

Cloning into 'glove'...
remote: Counting objects: 431, done.
^Cceiving objects:  80% (345/431), 164.01 KiB | 0 bytes/s   


In [ ]:
! make -C glove/

In [2]:
!ls glove/build/

cooccur  glove	shuffle  vocab_count


准备训练的语料

In [21]:
f  = open('corpus.txt','w')
f.write('XiaoMing like eating apple\n')
f.write('XiaoHong like eating banana\n')
f.close()

使用glove训练

In [24]:
!./glove/build/vocab_count -verbose 1 -max-vocab 20 -min-count 1 < corpus.txt > /tmp/vocab.txt

BUILDING VOCABULARY
Using vocabulary of size 6.



In [25]:
!./glove/build/cooccur -verbose 1 -symmetric 0 -window-size 3 -vocab-file /tmp/vocab.txt -memory 1.0 -overflow-file /tmp/tempoverflow < corpus.txt > /tmp/cooccurrence.bin

COUNTING COOCCURRENCES
window size: 3
context: asymmetric
Merging cooccurrence files: processed 11 lines.



In [26]:
!./glove/build/shuffle -verbose 1 -temp-file /tmp/tmp -memory 1.0 < /tmp/cooccurrence.bin > /tmp/cooccurrence.shuf.bin

SHUFFLING COOCCURRENCES
array size: 63753420
Segmentation fault (core dumped)


因为shuffle一直报错，所以训练用cooccurrence.bin代替了

In [30]:
!./glove/build/glove -verbose 2 -input-file /tmp/cooccurrence.bin -vocab-file /tmp/vocab.txt -save-file vectors -binary 0 -model 2 -vector-size 10 -threads 1 -alpha 0.75 -x-max 100.0 -eta 0.05 -iter 10

TRAINING MODEL
Read 11 lines.
Initializing parameters...done.
vector size: 10
vocab size: 6
x_max: 100.000000
alpha: 0.750000
07/30/18 - 08:38.01AM, iter: 001, cost: 0.005658
07/30/18 - 08:38.01AM, iter: 002, cost: 0.005637
07/30/18 - 08:38.01AM, iter: 003, cost: 0.005617
07/30/18 - 08:38.01AM, iter: 004, cost: 0.005597
07/30/18 - 08:38.01AM, iter: 005, cost: 0.005577
07/30/18 - 08:38.01AM, iter: 006, cost: 0.005557
07/30/18 - 08:38.01AM, iter: 007, cost: 0.005537
07/30/18 - 08:38.01AM, iter: 008, cost: 0.005517
07/30/18 - 08:38.01AM, iter: 009, cost: 0.005498
07/30/18 - 08:38.01AM, iter: 010, cost: 0.005479


In [31]:
!ls

corpus.txt  glove  Untitled.ipynb  vectors.txt


Generate Vocab from Glove Vectors.txt

In [32]:
import numpy as np
class Vocab(object):
    def __init__(self, lower=True):

        self.id2token = {}
        self.token2id = {}
        self.token_cnt = {}
        self.lower = lower

        self.vocab_size = None
        self.embeddings = None

    def prepare(self, filename):
        '''
        准备符号和id对应的字典
        '''
        tokens = []
        embs = []
        with open(filename, 'rb') as f:
            content = f.readlines()
            for line in content:
                # x = line
                line = line.split()
                # if len(line) != 201: # filter the bad line
                #    print(x.encode())
                #    continue
                tokens.append(line[0])
                embs.append(line[1:])

        ids = list(range(len(tokens)))
        self.token2id = dict(zip(tokens, ids))
        self.id2token = dict(zip(ids, tokens))

        self.embeddings = np.array(embs, dtype=np.float32)
        self.vocab_size = self.embeddings.shape[0]

    def get_id(self, token):
        '''
        得到一个符号的id
        '''
        token = token.encode('utf-8')
        token = token.lower() if self.lower else token
        try:
            return self.token2id[token]
        except KeyError:
            return self.token2id['<unk>']

    def convert_to_ids(self, tokens):
        '''
        符号列表转为id列表
        '''
        vec = [self.get_id(label) for label in tokens]
        return vec

In [33]:
vocab_1  =Vocab()

In [35]:
vocab_1.prepare('vectors.txt')